<img src="http://oci02.img.iteso.mx/identidad_de_instancia_2018/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg" width="800px" height="400px"/>

# <font color='midnightblue'> <center> Concurso Interuniversitario BMV-Peeptrade </center>

# <font color='midnightblue'> **Integrantes:**

<font color='midnightblue'> **Rubén Hernández Guevara | if717710**
    
<font color='midnightblue'> **Juan Pablo Rodríguez Alonso | if715180**
    
<font color='midnightblue'> **Andrés Ramírez Villanueva | if726061**

In [1]:
import matplotlib
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.express as px
import scipy.optimize as opt
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pandas_datareader.data as web
from plotly.subplots import make_subplots
from pandas_datareader import data as pdr
import yfinance as yfin
yfin.pdr_override()

In [2]:
rf = .05

In [3]:
prices = pd.read_csv('price_pfolio_1.csv')

In [4]:
def simulacion_pfolios(ret: 'Retornos',
                       rf: 'Tasa Libre de Riesgo',
                       g: 'Grado de Aversión al Riesgo'):
    
    global pesos, summary, w_EMV, w_opt, E_EMV, s_EMV
        
    #Rendimientos y volatilidades esperadas
    summary = pd.DataFrame(data={'Media': 252*ret.mean(),
                                 'Vol':((252)**(.5))*ret.std()})
        
    cov_ret = ret.cov() # Matriz de varianza - covarianza
    
    corr = ret.corr() # Matriz de correlación
    
    S = np.diag(summary['Vol'].values)
    Sigma = S.dot(corr).dot(S)
    Eind = summary['Media'].values
    
    n = len(Eind) # Número de activos    
    w0 = np.ones((n,))/n # Dato inicial   
    bnds = ((0, 1),)*n # Cotas de las variables    
    cons={'type':'eq','fun':lambda w:w.sum()-1} # Restricciones
    
    # Portafolio de mínima varianza
    minvar = minimize(fun=varianza,
                      x0=w0,
                      args=(Sigma,),
                      bounds=bnds,
                      constraints=cons,
                      tol=1e-5)
    
    w_minvar = minvar.x
    
    E_minvar = Eind.T.dot(w_minvar)
    s_minvar = (w_minvar.T.dot(Sigma).dot(w_minvar))**.5
    RS_minvar = (E_minvar - rf)/s_minvar

    # Portafolio EMV
    EMV = minimize(fun = menos_RS,
                   x0 = w0,
                   args = (Eind, Sigma, rf),
                   bounds = bnds,
                   constraints = cons,
                   tol = 1e-5)
    w_EMV = EMV.x
    
    # Pesos, rendimiento y riesgo del portafolio EMV
    E_EMV = Eind.T.dot(w_EMV)
    s_EMV = (w_EMV.T.dot(Sigma).dot(w_EMV))**0.5
    RS_EMV = (E_EMV - rf)/s_EMV
    
    #Asignación de capital
    w_opt = (E_EMV - rf)/(g*s_EMV**2)
    
    pesos = [w_EMV[i]*w_opt for i in range(len(w_EMV))]
    pesos.append(1-w_opt)
    pesos = np.round(pesos, 4)
    
    tickers = 'VNQ VBK SPY VTI VGT IBM AAL NVDA PFE MSFT AAPL VWAGY NFLX MA KO Cetes'
    tickers = tickers.split(' ')
    
    a_c = pd.DataFrame(columns = ['Asignación de Capital'], index=[tickers])
    a_c['Asignación de Capital'] = pesos
    
    return a_c

In [5]:
def simulacion_inversion(inversion: 'Dinero Invertido',
                         precios: 'Precios del Portafolio',
                         pesos: 'Pesos de Activos en Portafolio',
                         retornos: 'Retornos del Portafolio'):
    
    global inv_real, inv_teor, asignacion_activos, asignacion_rf, vfinal_activos, vfinal_rf, vfinal_pfolio,\
    rnd_activos, rnd_rf, rnd_pfolio, cumrets, Resultados
    
    asignacion_activos = np.sum(pesos[0:15])*inversion
    asignacion_rf = inversion - asignacion_activos
    
    inv_teor = [pesos[i]*inversion for i in range(15)] # Inversión Teórica de Acuerdo a las Ponderaciones. 
    
    vi_acciones = [precios.iloc[0].values[i] for i in range(15)] # Valor Inicial Real de las Acciones Al Momento de la Inversión.
    
    # Número de acciones a comprar
    n_acciones = [(inv_teor[i]/vi_acciones[i]) for i in range(15)]
    n_acciones = np.round(n_acciones) 
    
    inv_real = [n_acciones[i]*vi_acciones[i] for i in range(15)] # Inversión real, de acuerdo a n acciones y precio inicial
    
    restante_inv_rf = [inv_teor[i]-inv_real[i] for i in range(15)]
    restante_inv_rf = np.sum(restante_inv_rf)
    asignacion_rf += restante_inv_rf
    
    
    cumrets = pd.DataFrame(data = {'Date': retornos['Date'],
                               'VNQ': (retornos['VNQ'] + 1).cumprod()*(inv_real[0]),
                               'VBK': (retornos['VBK'] + 1).cumprod()*(inv_real[1]),
                               'SPY': (retornos['SPY'] + 1).cumprod()*(inv_real[2]),
                               'VTI': (retornos['VTI'] + 1).cumprod()*(inv_real[3]),
                               'VGT': (retornos['VGT'] + 1).cumprod()*(inv_real[4]),
                               'IBM': (retornos['IBM'] + 1).cumprod()*(inv_real[5]),
                               'AAL': (retornos['AAL'] + 1).cumprod()*(inv_real[6]),
                               'NVDA': (retornos['NVDA'] + 1).cumprod()*(inv_real[7]),
                               'PFE': (retornos['PFE'] + 1).cumprod()*(inv_real[8]),
                               'MSFT': (retornos['MSFT'] + 1).cumprod()*(inv_real[9]),
                               'AAPL': (retornos['AAPL'] + 1).cumprod()*(inv_real[10]),
                               'VWAGY': (retornos['VWAGY'] + 1).cumprod()*(inv_real[11]),
                               'NFLX': (retornos['NFLX'] + 1).cumprod()*(inv_real[12]),
                               'MA': (retornos['MA'] + 1).cumprod()*(inv_real[13]),
                               'KO': (retornos['KO'] + 1).cumprod()*(inv_real[14])})

    
    fig = px.line(cumrets, x = "Date", y = cumrets.columns,
              hover_data = {"Date": "|%B %d, %Y"},
              title = 'Evolución del Portafolio')
    fig.update_xaxes(
        dtick = "M1",
        tickformat="%b\n%Y")
    fig.show()
    
    # Valor Final de Inversión 
    vfinal_activos = np.sum(cumrets.tail(1).values[0][1:])
    vfinal_rf = asignacion_rf*(1+rf)   
    vfinal_pfolio = vfinal_activos + vfinal_rf

    # RETORNO DE ACCIONES %
    rnd_activos = pct_chng(vfinal_activos, np.sum(inv_real))
    rnd_rf = pct_chng(vfinal_rf, asignacion_rf)
    rnd_pfolio = pct_chng(vfinal_pfolio, inversion)
    
    Resultados = pd.DataFrame(columns=['Tipo', 'Inversión Inicial', 'Valor Final', 'Rendimiento %'])
    
    Resultados['Tipo'] = ['Activos','Tasa', 'Portafolio']
    Resultados['Inversión Inicial'] = [np.sum(inv_real), asignacion_rf, inversion]
    Resultados['Valor Final'] = [vfinal_activos, vfinal_rf, vfinal_pfolio]
    Resultados['Rendimiento %'] = [rnd_activos, rnd_rf, rnd_pfolio]
    
    return np.round(Resultados, 4)

    inv_real, asignacion_rf, vfinal_activos, vfinal_rf, vfinal_pfolio, rnd_activos, rnd_rf, rnd_pfolio, cumrets = None

In [6]:
# Cambiar estilo de gráficos a seaborn
matplotlib.style.use('seaborn')

# Cambiar motor gráfico a PLOTLY
pd.options.plotting.backend = "plotly"

In [7]:
# Función objetivo
def varianza(w,Sigma):
    return w.T.dot(Sigma).dot(w) 

In [8]:
# Función objetivo
def menos_RS(w,Eind,Sigma,rf):
    Ep = Eind.T.dot(w)
    sp = (w.T.dot(Sigma).dot(w))**.5
    RS = (Ep-rf)/sp
    return -RS

In [9]:
# Función Cambio Porcentual
def pct_chng(VA: 'Valor Actual',
             VI: 'Valor Inicial'):  
    ret = np.round(((VA-VI)/VI), 2)
    return ret

In [10]:
prices

,VNQ,VBK,SPY,VTI,VGT,IBM,AAL,NVDA,PFE,MSFT,AAPL,VWAGY,NFLX,MA,KO
0,62.548244,114.420456,180.745560,92.627029,99.869308,105.509613,39.095459,7.912141,24.401283,49.681786,24.286829,14.313305,109.959999,91.394379,35.161503
1,63.755211,114.468430,181.051270,92.834396,99.391563,105.431984,38.722755,8.039244,24.576944,49.908424,23.678217,13.476381,107.660004,91.567917,35.285900
2,63.580544,112.683189,178.767441,91.590218,98.042610,104.904259,39.401276,7.706822,24.141613,49.001827,23.214844,13.381275,117.680000,90.167366,35.095169
3,62.341812,109.515747,174.478531,89.345306,94.885674,103.111465,38.655865,7.401288,23.981228,47.297417,22.235073,12.767849,114.559998,88.515648,34.514675
4,61.508038,107.912857,172.563339,88.326523,94.070686,102.156876,38.579403,7.242407,23.675735,47.442482,22.352644,12.985640,111.389999,86.825302,34.423447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,67.314377,130.247986,207.303436,107.098373,116.999908,134.312347,46.847836,27.091278,25.744190,58.906742,27.459555,14.049389,125.589996,101.753822,35.618557
248,67.455963,130.888596,207.817780,107.365128,117.588867,134.658798,46.973461,28.951988,25.783823,58.944000,27.633949,14.023234,128.350006,101.841286,35.627121
249,67.047836,129.403519,206.100250,106.472816,116.410980,133.893417,46.065105,26.960493,25.641153,58.673870,27.516119,13.823514,125.889999,100.752899,35.438755
250,67.880745,129.675339,206.054367,106.482010,116.477478,134.223724,45.659245,27.498468,25.752123,58.590046,27.509050,13.690368,125.330002,100.840355,35.618557


In [11]:
ret = prices.pct_change()

In [12]:
ret.head()

,VNQ,VBK,SPY,VTI,VGT,IBM,AAL,NVDA,PFE,MSFT,AAPL,VWAGY,NFLX,MA,KO
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.019297,0.000419,0.001691,0.002239,-0.004784,-0.000736,-0.009533,0.016064,0.007199,0.004562,-0.025059,-0.058472,-0.020917,0.001899,0.003538
2,-0.002740,-0.015596,-0.012614,-0.013402,-0.013572,-0.005005,0.017523,-0.041350,-0.017713,-0.018165,-0.019570,-0.007057,0.093071,-0.015295,-0.005405
3,-0.019483,-0.028109,-0.023992,-0.024510,-0.032200,-0.017090,-0.018918,-0.039645,-0.006644,-0.034783,-0.042204,-0.045842,-0.026513,-0.018318,-0.016541
4,-0.013374,-0.014636,-0.010977,-0.011403,-0.008589,-0.009258,-0.001978,-0.021467,-0.012739,0.003067,0.005288,0.017058,-0.027671,-0.019097,-0.002643


In [13]:
#Rendimientos y volatilidades esperadas
summary = pd.DataFrame(data={'Media': 252*ret.mean(),
                             'Vol':((252)**(.5))*ret.std()})

In [14]:
summary.sort_values(by =['Media'], ascending=False)

,Media,Vol
NVDA,1.312650,0.470650
IBM,0.257630,0.197548
NFLX,0.217263,0.444719
AAL,0.210708,0.365627
MSFT,0.179124,0.227002
VGT,0.158882,0.165016
VTI,0.145246,0.134261
AAPL,0.144534,0.233849
VBK,0.137413,0.174437
SPY,0.136368,0.130063


In [15]:
cov_ret = ret.cov() # Matriz de varianza - covarianza
cov_ret

,VNQ,VBK,SPY,VTI,VGT,IBM,AAL,NVDA,PFE,MSFT,AAPL,VWAGY,NFLX,MA,KO
VNQ,0.000113,0.000072,0.000051,0.000054,0.000057,0.000047,0.000058,0.000078,0.000027,0.000058,0.000047,0.000045,0.000049,0.000056,0.000048
VBK,0.000072,0.000121,0.000081,0.000087,0.000097,0.000071,0.000142,0.000136,0.000057,0.000094,0.000082,0.000110,0.000102,0.000097,0.000036
SPY,0.000051,0.000081,0.000067,0.000069,0.000077,0.000063,0.000109,0.000088,0.000048,0.000083,0.000069,0.000084,0.000084,0.000077,0.000038
VTI,0.000054,0.000087,0.000069,0.000072,0.000080,0.000064,0.000114,0.000094,0.000049,0.000084,0.000070,0.000088,0.000086,0.000080,0.000037
VGT,0.000057,0.000097,0.000077,0.000080,0.000108,0.000075,0.000133,0.000147,0.000045,0.000122,0.000108,0.000103,0.000125,0.000104,0.000040
IBM,0.000047,0.000071,0.000063,0.000064,0.000075,0.000155,0.000108,0.000113,0.000036,0.000077,0.000056,0.000096,0.000059,0.000069,0.000032
AAL,0.000058,0.000142,0.000109,0.000114,0.000133,0.000108,0.000530,0.000209,0.000070,0.000134,0.000109,0.000197,0.000190,0.000147,0.000046
NVDA,0.000078,0.000136,0.000088,0.000094,0.000147,0.000113,0.000209,0.000879,0.000013,0.000147,0.000124,0.000125,0.000152,0.000149,0.000054
PFE,0.000027,0.000057,0.000048,0.000049,0.000045,0.000036,0.000070,0.000013,0.000139,0.000043,0.000038,0.000043,0.000039,0.000046,0.000014
MSFT,0.000058,0.000094,0.000083,0.000084,0.000122,0.000077,0.000134,0.000147,0.000043,0.000204,0.000104,0.000097,0.000129,0.000115,0.000054


In [16]:
corr = ret.corr() # Matriz de correlación
corr

,VNQ,VBK,SPY,VTI,VGT,IBM,AAL,NVDA,PFE,MSFT,AAPL,VWAGY,NFLX,MA,KO
VNQ,1.000000,0.618132,0.591019,0.602908,0.515859,0.355019,0.238308,0.248859,0.218609,0.383536,0.303064,0.209129,0.163559,0.401399,0.509254
VBK,0.618132,1.000000,0.903832,0.933868,0.849918,0.522376,0.559531,0.417953,0.439128,0.595862,0.507146,0.493831,0.329863,0.678349,0.368969
SPY,0.591019,0.903832,1.000000,0.995840,0.906807,0.613646,0.578849,0.360464,0.491889,0.705430,0.569305,0.503898,0.366255,0.723799,0.514623
VTI,0.602908,0.933868,0.995840,1.000000,0.905707,0.608679,0.585660,0.376434,0.488164,0.695213,0.561135,0.511188,0.360954,0.720388,0.490936
VGT,0.515859,0.849918,0.906807,0.905707,1.000000,0.579478,0.555161,0.478480,0.363538,0.819638,0.707860,0.487332,0.429615,0.764858,0.430452
IBM,0.355019,0.522376,0.613646,0.608679,0.579478,1.000000,0.377392,0.307084,0.242224,0.431958,0.305428,0.379387,0.170382,0.427321,0.286184
AAL,0.238308,0.559531,0.578849,0.585660,0.555161,0.377392,1.000000,0.305593,0.256958,0.407284,0.320515,0.421351,0.295115,0.487060,0.223531
NVDA,0.248859,0.417953,0.360464,0.376434,0.478480,0.307084,0.305593,1.000000,0.035869,0.347017,0.283269,0.208761,0.182429,0.383663,0.202641
PFE,0.218609,0.439128,0.491889,0.488164,0.363538,0.242224,0.256958,0.035869,1.000000,0.254029,0.219803,0.179762,0.118196,0.298979,0.129579
MSFT,0.383536,0.595862,0.705430,0.695213,0.819638,0.431958,0.407284,0.347017,0.254029,1.000000,0.495037,0.336464,0.321759,0.616454,0.420100


In [17]:
S = np.diag(summary['Vol'].values)

In [18]:
Sigma = S.dot(corr).dot(S)

In [19]:
Eind = summary['Media'].values

In [20]:
n = len(Eind) # Número de activos    
w0 = np.ones((n,))/n # Dato inicial   
bnds = ((0, .1),)*n # Cotas de las variables    
cons={'type':'eq','fun':lambda w:w.sum()-1} # Restricciones


In [21]:
# Portafolio de mínima varianza
minvar = minimize(fun=varianza,
                  x0=w0,
                  args=(Sigma,),
                  bounds=bnds,
                  constraints=cons,
                  tol=1e-5)

w_minvar = minvar.x

In [22]:
# Rendimiento del Portafolio con Mínima Varianza
E_minvar = Eind.T.dot(w_minvar)
E_minvar 

0.12859963610889305

In [23]:
# Volatilidad del Portafolio con Mínima Varianza
s_minvar = (w_minvar.T.dot(Sigma).dot(w_minvar))**.5
s_minvar

0.12950659319114838

In [24]:
# Ratio de Sharpe con Portafolio de Mínima Varianza
RS_minvar = (E_minvar - rf)/s_minvar
RS_minvar

0.6069160972590949

# Portafolio en Media Varianza con Máximo RS

In [25]:
# Portafolio EMV
EMV = minimize(fun = menos_RS,
               x0 = w0,
               args = (Eind, Sigma, rf),
               bounds = bnds,
               constraints = cons)
w_EMV = EMV.x

In [26]:
# Rendimiento del Portafolio con Media Varianza
E_EMV = Eind.T.dot(w_EMV)
E_EMV

0.2652967779395502

In [27]:
# Volatilidad del Portafolio con Media Varianza
s_EMV = (w_EMV.T.dot(Sigma).dot(w_EMV))**0.5
s_EMV

0.14837327029478917

In [28]:
# Ratio de Sharpe con Portafolio de Mínima Varianza
RS_EMV = (E_EMV - rf)/s_EMV
RS_EMV

1.4510482751495395

In [29]:
np.round(w_EMV, 4)

array([0.1   , 0.0499, 0.1   , 0.1   , 0.1   , 0.1   , 0.    , 0.1   ,
       0.1   , 0.1   , 0.0871, 0.    , 0.0386, 0.    , 0.0245])

In [30]:
summary['w'] = np.round(w_EMV,4)

In [31]:
summary

,Media,Vol,w
VNQ,0.108959,0.168619,0.1000
VBK,0.137413,0.174437,0.0499
SPY,0.136368,0.130063,0.1000
VTI,0.145246,0.134261,0.1000
VGT,0.158882,0.165016,0.1000
IBM,0.257630,0.197548,0.1000
AAL,0.210708,0.365627,0.0000
NVDA,1.312650,0.470650,0.1000
PFE,0.071064,0.187145,0.1000
MSFT,0.179124,0.227002,0.1000
